# Name: Sakshi Modi
# Reg No: 21112022
# Class: 5 BSc DS

## CAC 1 Component 2: Minesweeper Game Enhancements

The Minesweeper game has been enhanced to improve the user experience and align it with traditional Minesweeper gameplay. The following features have been added and adjustments made:

1. **Triggered Bombs:** The updated version differentiates triggered (detonated) bombs by coloring them green. This visual distinction allows players to identify which bombs caused them to lose the game, providing a clearer understanding of the game over scenario.

2. **Resizable Grid:** This improved version introduces dynamic grid resizing based on the window dimensions. When the player resizes the game window, the grid adapts to fit within the visible area. This feature ensures a user-friendly experience and accommodates various screen sizes.

3. **Right-Click Flagging:** In this updated game, you can only place a flag on a cell if it is not already visible. This behavior aligns with the traditional Minesweeper rules. Right-clicking a flagged cell toggles the flag state, and the menu bar displays the total flag count. This contrasts with the original code, which allowed flagging on any cell, whether visible or not.

4. **"Game Over" and "You Won!" Messages:** The enhanced game presents "Game Over" and "You Won!" messages in the menu bar upon game completion. These messages offer players immediate feedback regarding the game's outcome.

5. **Elimination of the Initial Click on a Bomb:** In the updated game, the initial click on a bomb is prevented. Bombs are placed only after the player's first move, ensuring that players cannot lose the game with their initial action.

6. **Color Changes:** The code introduces adjustments to cell colors, making it easier to distinguish different states, such as triggered bombs displayed in green.

## Approach to Building the Game

The Minesweeper game was enhanced using the following approach:

1. **Understanding the Original Code:** The first step was to thoroughly understand the original Minesweeper game code and its mechanics.

2. **Identifying Areas for Improvement:** We identified areas where the game could be enhanced to improve user experience and align it with traditional Minesweeper gameplay.

3. **Implementing Feature Additions:** We introduced new features such as triggered bomb highlighting, resizable grid, right-click flagging, and game outcome messages.

4. **Eliminating Initial Click on Bomb:** To prevent players from losing on their first move, we ensured that bombs are placed after the initial click.

5. **Enhancing Visuals:** Visual cues, such as color changes for cell states, were implemented to improve game clarity.

These changes collectively make the Minesweeper game more engaging, user-friendly, and faithful to the traditional Minesweeper experience.


In [2]:
import pygame
import sys

from random import randrange

# Colors used
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
GRAY = (127, 127, 127)
# Sets the WIDTH and HEIGHT of each grid location
WIDTH = 30
HEIGHT = 30
# Sets the starting number of squares
NSQUARES = 10
# Sets the margin between each cell
MARGIN = 5
MENU_SIZE = 40
LEFT_CLICK = 1
RIGHT_CLICK = 3


# Class that holds the game logic          
class Game:
    
    
    def __init__(self):
        # Create a grid of NSQUARES x NSQUARES
        self.grid = [[self.Cell(x, y) for x in range(NSQUARES)] for y in range(NSQUARES)]
        self.init = False
        self.game_lost = False
        self.game_won = False
        self.num_bombs = 10
        self.squares_x = NSQUARES
        self.squares_y = NSQUARES
        self.resize = False
        self.flag_count = 0

    def draw(self):
        # Set the screen background color
        screen.fill(BLACK)
        # Draw the grid
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                color = WHITE
                if self.grid[row][column].is_visible:
                     color = RED if self.grid[row][column].has_bomb else GRAY  
                elif self.grid[row][column].has_flag:
                    color = BLUE
                pygame.draw.rect(screen,
                                color,
                                [(MARGIN + WIDTH) * column + MARGIN,
                                (MARGIN + HEIGHT) * row + MARGIN + MENU_SIZE,
                                WIDTH,
                                HEIGHT])
                self.grid[row][column].show_text()
        
    # Adjusts the grid when the screen size has changed
    def adjust_grid(self, sizex, sizey):
        global screen
        self.squares_x = (sizex - MARGIN) // (WIDTH + MARGIN)
        self.squares_y = (sizey - MARGIN - MENU_SIZE) // (HEIGHT + MARGIN)
        if self.squares_x < 8:
            self.squares_x = 8
        if self.squares_y < 8:
            self.squares_y = 8
        if self.num_bombs > (self.squares_x * self.squares_y) // 3:
            self.num_bombs = self.squares_x * self.squares_y // 3
        self.grid = [[self.Cell(x, y) for x in range(self.squares_x)] for y in range(self.squares_y)]
        size = ((self.squares_x*(WIDTH + MARGIN) + MARGIN), (self.squares_y*(HEIGHT + MARGIN) + MARGIN + MENU_SIZE))
        screen = pygame.display.set_mode(size, pygame.RESIZABLE)

    # Makes all cells visible when user loses
    def game_over(self):
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                if self.grid[row][column].has_bomb:
                    self.grid[row][column].is_visible = True
                self.grid[row][column].has_flag = False

    # Changes the number of bombs placed and caps it
    def change_num_bombs(self, bombs):
        self.num_bombs += bombs
        if self.num_bombs < 1:
            self.num_bombs = 1
        elif self.num_bombs > (self.squares_x * self.squares_y) // 3:
            self.num_bombs = self.squares_x * self.squares_y // 3
        self.reset_game() 

    # Place BOMBS on random places
    def place_bombs(self, row, column):
        bombplaced = 0
        while bombplaced < self.num_bombs:
            x = randrange(self.squares_y)
            y = randrange(self.squares_x)
            if not self.grid[x][y].has_bomb and not (row == x and column == y):
                self.grid[x][y].has_bomb = True
                bombplaced += 1
        self.count_all_bombs()
        if self.grid[row][column].bomb_count != 0:
            self.reset_game()
            self.place_bombs(row, column)
        
    # Count all bombs next to a cell (3x3) for the entire grid
    def count_all_bombs(self):
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                self.grid[row][column].count_bombs(self.squares_y, self.squares_x)
    
    # Restarts the game
    def reset_game(self):
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                self.init = False
                self.grid[row][column].is_visible = False
                self.grid[row][column].has_bomb = False
                self.grid[row][column].bomb_count = 0
                self.grid[row][column].test = False
                self.grid[row][column].has_flag = False
                self.game_lost = False
                self.game_won = False
                self.flag_count = 0

    def check_victory(self):   
        count = 0
        total = self.squares_x * self.squares_y
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                if self.grid[row][column].is_visible:
                    count += 1
        if ((total - count) == self.num_bombs) and not self.game_lost:
            self.game_won = True
            for row in range(self.squares_y):
                for column in range(self.squares_x):
                    if self.grid[row][column].has_bomb:
                        self.grid[row][column].has_flag = True
        
    
    def count_flags(self):
        total_flags = 0
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                if self.grid[row][column].has_flag:
                            total_flags += 1
        self.flag_count = total_flags

    # Handle for grid clicks
    def click_handle(self, row, column, button):
        if button == LEFT_CLICK and self.game_won:
                self.reset_game()
        elif button == LEFT_CLICK and not self.grid[row][column].has_flag: 
            if not self.game_lost:
                # Place bombs after first click so you never click a bomb first
                if not self.init:
                    self.place_bombs(row, column)
                    self.init = True
                # Set the click square to visible
                self.grid[row][column].is_visible = True
                self.grid[row][column].has_flag = False
                if self.grid[row][column].has_bomb:
                    self.game_over()
                    self.game_lost = True
                if self.grid[row][column].bomb_count == 0 and not self.grid[row][column].has_bomb:
                    self.grid[row][column].open_neighbours(self.squares_y, self.squares_x)
                self.check_victory()
            else:
                self.game_lost = False
                self.reset_game()
        
        elif button == RIGHT_CLICK and not self.game_won:
            if not self.grid[row][column].has_flag:
                if self.flag_count < self.num_bombs and not self.grid[row][column].is_visible:
                    self.grid[row][column].has_flag = True
            else:
                self.grid[row][column].has_flag = False
            self.count_flags()


    # Game Sub-Class for each Cell of the grid
    class Cell:

        def __init__(self, x, y):
            self.x = x
            self.y = y
            self.is_visible = False
            self.has_bomb = False
            self.bomb_count = 0
            self.text = ""
            self.test = False
            self.has_flag = False

        # Handle for the number of bombs text
        def show_text(self):
            if self.is_visible:
                if self.bomb_count == 0:
                    self.text = font.render("", True, BLACK)
                else:
                    self.text = font.render(str(self.bomb_count), True, BLACK)
                screen.blit(self.text, (self.x * (WIDTH + MARGIN) + 12, self.y * (HEIGHT + MARGIN) + 10 + MENU_SIZE))
        
        # Counts how many bombs are next to this cell (3x3)
        def count_bombs(self, squaresx, squaresy):
            if not self.test:
                self.test = True
                if not self.has_bomb:
                    for column in range(self.x - 1 , self.x + 2):
                        for row in range(self.y - 1 , self.y + 2):
                            if (row >= 0 and row < squaresx and column >= 0 and column < squaresy
                                and not (column == self.x and row == self.y)
                                and game.grid[row][column].has_bomb):
                                    self.bomb_count += 1
        
        # Open all cells next to the clicked cell with zero bombs nearby
        def open_neighbours(self, squaresx, squaresy):
            column = self.x
            row = self.y
            for row_off in range(-1, 2):
                for column_off in range(-1, 2):
                    if ((row_off == 0 or column_off == 0) and row_off != column_off
                        and row+row_off >= 0 and column+column_off >=0 and row+row_off < squaresx and column+column_off < squaresy):
                            game.grid[row + row_off][column + column_off].count_bombs(game.squares_y, game.squares_x)
                            if not game.grid[row + row_off][column + column_off].is_visible and not game.grid[row + row_off][column + column_off].has_bomb:  
                                    game.grid[row + row_off][column + column_off].is_visible = True
                                    game.grid[row + row_off][column + column_off].has_flag = False
                                    if game.grid[row + row_off][column + column_off].bomb_count == 0: 
                                        game.grid[row + row_off][column + column_off].open_neighbours(game.squares_y, game.squares_x)


class Menu():

    def __init__(self):
        self.width = pygame.display.get_surface().get_width() - 2*MARGIN
        self.btn_minus = self.Button(10, 10, 20, 20, "-", 6, -3)
        self.btn_plus = self.Button(60, 10, 20, 20, "+", 3, -4)
        self.btn_flags = self.Button(280, 16, 10, 10, "")
        self.btn_flags.background = BLUE
        self.label_bombs = self.Label(30, 10)
        self.label_game_end = self.Label(100, 10)
        self.label_flags = self.Label(self.width - 50, 10)

    def click_handle(self, obj):
        if self.btn_minus.click_handle():
            obj.change_num_bombs(-1)
        if self.btn_plus.click_handle():
            obj.change_num_bombs(1)
        
    def draw(self, obj):
        self.width = pygame.display.get_surface().get_width() - 2*MARGIN 
        pygame.draw.rect(screen, GRAY, [MARGIN, 0, self.width, MENU_SIZE])
        self.btn_minus.draw(screen)
        self.btn_plus.draw(screen)
        self.btn_flags.draw(screen)
        self.label_bombs.show(screen, game.num_bombs)
        self.label_flags.show(screen, game.flag_count)
        if obj.game_lost:
            self.label_game_end.show(screen, "Game Over")
        elif obj.game_won:
            self.label_game_end.show(screen, "You Won!")
    

    class Label:
    
        def __init__(self, x, y):
            self.x = x
            self.y = y
            self.text = ""
        
        def show(self, surface, value): 
            text = str(value)
            self.text = font.render(text, True, BLACK)     
            surface.blit(self.text, (self.x, self.y))
    

    class Button:

        def __init__(self, x, y, width, height, text, xoff=0, yoff=0):
            self.x = x
            self.y = y
            self.height = height
            self.width = width
            self.background = WHITE
            self.text = text
            self.x_offset = xoff
            self.y_offset = yoff

        def draw(self, surface):
            pygame.draw.ellipse(surface, self.background, [self.x, self.y, self.width, self.height], 0)
            text = font.render(self.text, True, BLACK)     
            surface.blit(text, (self.x + self.x_offset, self.y + self.y_offset))
        
        def click_handle(self):
            pos = pygame.mouse.get_pos()
            if pos[0] > self.x and pos[1] > self.y and pos[0] < (self.x + self.width) and pos[1] < (self.y + self.height):
                return True
            else:
                return False


# Initialize pygame and sets screen size and caption
pygame.init()
size = (NSQUARES*(WIDTH + MARGIN) + MARGIN, (NSQUARES*(HEIGHT + MARGIN) + MARGIN) + MENU_SIZE)
screen = pygame.display.set_mode(size, pygame.RESIZABLE)
pygame.display.set_caption("Minesweeper")
# Font to use in the entire game
font = pygame.font.Font('freesansbold.ttf', 24)
# Create instances for Game and Menu
game = Game()
menu = Menu()
clock = pygame.time.Clock()
# Main loop
while True:
    for event in pygame.event.get():
        # Closes the game if user clicked the X
        if event.type == pygame.QUIT:  
            pygame.quit()
            sys.exit()
        # Mouse clicks event
        elif event.type == pygame.MOUSEBUTTONDOWN:
                # Get mouse position
                position = pygame.mouse.get_pos()
                # Change the screen coordinates to grid coordinates and caps max values
                column = position[0] // (WIDTH + MARGIN)
                row = (position[1] - MENU_SIZE) // (HEIGHT + MARGIN)
                if row >= game.squares_y:
                    row = game.squares_y - 1
                if column >= game.squares_x:
                    column = game.squares_x - 1
                if row >= 0:
                    game.click_handle(row, column, event.button)
                else:
                    menu.click_handle(game)
        # Event for screen resize    
        elif event.type == pygame.VIDEORESIZE:
            if game.resize: 
                game.adjust_grid(event.w, event.h)
                game.reset_game()
            else:  
                game.resize = True
    game.draw()
    menu.draw(game)
    clock.tick(60)
    # Update the screen
    pygame.display.flip()

SystemExit: 

# The updated one!
|   | Feature Description                                 |
|---|-------------------------------------------------------|
| 1 | An amalgamation of memory and Minesweeper            |
| 2 | Triggered bombs                                      |
| 3 | If you click a cell and no response, there is a bomb that you have to remember  |
| 4 | There will be fewer cells with numbers and more cells with grey blocks   |


In [8]:
import pygame
import sys
from random import randrange

# Colors used
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
GRAY = (127, 127, 127)

# Sets the WIDTH and HEIGHT of each grid location
WIDTH = 30
HEIGHT = 30
NSQUARES = 10
MARGIN = 5
MENU_SIZE = 40
LEFT_CLICK = 1
RIGHT_CLICK = 3

# Class that holds the game logic
class Game:
    def __init__(self):
        self.grid = [[self.Cell(x, y) for x in range(NSQUARES)] for y in range(NSQUARES)]
        self.init = False
        self.game_lost = False
        self.game_won = False
        self.num_bombs = 10
        self.squares_x = NSQUARES
        self.squares_y = NSQUARES
        self.resize = False
        self.flag_count = 0

    def draw(self):
        screen.fill(BLACK)
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                color = WHITE
                if self.grid[row][column].is_visible:
                    if self.grid[row][column].triggered:
                        color = GREEN  # Display triggered bombs differently
                    else:
                        color = RED if self.grid[row][column].has_bomb else GRAY
                elif self.grid[row][column].has_flag:
                    color = BLUE
                pygame.draw.rect(screen,
                                color,
                                [(MARGIN + WIDTH) * column + MARGIN,
                                (MARGIN + HEIGHT) * row + MARGIN + MENU_SIZE,
                                WIDTH,
                                HEIGHT])
                self.grid[row][column].show_text()

    def adjust_grid(self, sizex, sizey):
        global screen
        self.squares_x = (sizex - MARGIN) // (WIDTH + MARGIN)
        self.squares_y = (sizey - MARGIN - MENU_SIZE) // (HEIGHT + MARGIN)
        if self.squares_x < 8:
            self.squares_x = 8
        if self.squares_y < 8:
            self.squares_y = 8
        if self.num_bombs > (self.squares_x * self.squares_y) // 3:
            self.num_bombs = self.squares_x * self.squares_y // 3
        self.grid = [[self.Cell(x, y) for x in range(self.squares_x)] for y in range(self.squares_y)]
        size = ((self.squares_x * (WIDTH + MARGIN) + MARGIN), (self.squares_y * (HEIGHT + MARGIN) + MARGIN + MENU_SIZE))
        screen = pygame.display.set_mode(size, pygame.RESIZABLE)

    def game_over(self):
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                if self.grid[row][column].has_bomb:
                    self.grid[row][column].is_visible = True
                self.grid[row][column].has_flag = False

    def change_num_bombs(self, bombs):
        self.num_bombs += bombs
        if self.num_bombs < 1:
            self.num_bombs = 1
        elif self.num_bombs > (self.squares_x * self.squares_y) // 3:
            self.num_bombs = self.squares_x * self.squares_y // 3
        self.reset_game()

    def place_bombs(self, row, column):
        bombplaced = 0
        while bombplaced < self.num_bombs:
            x = randrange(self.squares_y)
            y = randrange(self.squares_x)
            if not self.grid[x][y].has_bomb and not (row == x and column == y):
                self.grid[x][y].has_bomb = True
                bombplaced += 1
        self.count_all_bombs()
        if self.grid[row][column].bomb_count != 0:
            self.reset_game()
            self.place_bombs(row, column)

    def count_all_bombs(self):
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                self.grid[row][column].count_bombs(self.squares_y, self.squares_x)

    def reset_game(self):
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                self.init = False
                self.grid[row][column].is_visible = False
                self.grid[row][column].has_bomb = False
                self.grid[row][column].bomb_count = 0
                self.grid[row][column].test = False
                self.grid[row][column].has_flag = False
                self.grid[row][column].triggered = False  # Reset triggered bombs
                self.game_lost = False
                self.game_won = False
                self.flag_count = 0

    def check_victory(self):
        count = 0
        total = self.squares_x * self.squares_y
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                if self.grid[row][column].is_visible:
                    count += 1
        if ((total - count) == self.num_bombs) and not self.game_lost:
            self.game_won = True
            for row in range(self.squares_y):
                for column in range(self.squares_x):
                    if self.grid[row][column].has_bomb:
                        self.grid[row][column].has_flag = True

    def count_flags(self):
        total_flags = 0
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                if self.grid[row][column].has_flag:
                    total_flags += 1
        self.flag_count = total_flags
        
    def click_handle(self, row, column, button):
        if button == LEFT_CLICK:
            if not self.game_won:
                if not self.grid[row][column].has_flag:
                    if not self.init:
                        self.place_bombs(row, column)
                        self.init = True
                    if self.grid[row][column].has_bomb:
                        if not self.grid[row][column].triggered:
                            self.grid[row][column].triggered = True
                        else:
                            self.grid[row][column].is_visible = True
                            self.game_over()
                            self.game_lost = True
                    else:
                        self.grid[row][column].is_visible = True
                        if self.grid[row][column].bomb_count == 0:
                            self.grid[row][column].open_neighbours(self.squares_y, self.squares_x)
                        self.check_victory()
            else:
                self.game_lost = False
                self.reset_game()
        elif button == RIGHT_CLICK and not self.game_won:
            if not self.grid[row][column].is_visible:
                if self.flag_count < self.num_bombs:
                    self.grid[row][column].has_flag = not self.grid[row][column].has_flag
                self.count_flags()

    class Cell:
        def __init__(self, x, y):
            self.x = x
            self.y = y
            self.is_visible = False
            self.has_bomb = False
            self.bomb_count = 0
            self.text = ""
            self.test = False
            self.has_flag = False
            self.triggered = False

        def show_text(self):
            if self.is_visible:
                if self.bomb_count == 0:
                    self.text = font.render("", True, BLACK)
                else:
                    self.text = font.render(str(self.bomb_count), True, BLACK)
                screen.blit(self.text, (self.x * (WIDTH + MARGIN) + 12, self.y * (HEIGHT + MARGIN) + 10 + MENU_SIZE))

        def count_bombs(self, squaresx, squaresy):
            if not self.test:
                self.test = True
                if not self.has_bomb:
                    for column in range(self.x - 1, self.x + 2):
                        for row in range(self.y - 1, self.y + 2):
                            if (row >= 0 and row < squaresx and column >= 0 and column < squaresy
                                    and not (column == self.x and row == self.y)
                                    and game.grid[row][column].has_bomb):
                                self.bomb_count += 1

        def open_neighbours(self, squaresx, squaresy):
            column = self.x
            row = self.y
            for row_off in range(-1, 2):
                for column_off in range(-1, 2):
                    if ((row_off == 0 or column_off == 0) and row_off != column_off
                            and row + row_off >= 0 and column + column_off >= 0 and row + row_off < squaresx and column + column_off < squaresy):
                        game.grid[row + row_off][column + column_off].count_bombs(game.squares_y, game.squares_x)
                        if not game.grid[row + row_off][column + column_off].is_visible and not game.grid[row + row_off][column + column_off].has_bomb:
                            game.grid[row + row_off][column + column_off].is_visible = True
                            game.grid[row + row_off][column + column_off].has_flag = False

class Menu():
    def __init__(self):
        self.width = pygame.display.get_surface().get_width() - 2 * MARGIN
        self.btn_minus = self.Button(10, 10, 20, 20, "-", 6, -3)
        self.btn_plus = self.Button(60, 10, 20, 20, "+", 3, -4)
        self.btn_flags = self.Button(280, 16, 10, 10, "")
        self.btn_flags.background = BLUE
        self.label_bombs = self.Label(30, 10)
        self.label_game_end = self.Label(100, 10)
        self.label_flags = self.Label(self.width - 50, 10)

    def click_handle(self, row, column, button):
        if button == LEFT_CLICK:
            if not self.game_won:
                if not self.grid[row][column].has_flag:
                    if not self.init:
                        self.place_bombs(row, column)
                        self.init = True
                    if self.grid[row][column].has_bomb:
                        if not self.grid[row][column].triggered:
                            self.grid[row][column].triggered = True
                        if not self.game_lost:
                            self.grid[row][column].is_visible = True
                            self.game_over()
                            self.game_lost = True
                    else:
                        self.grid[row][column].is_visible = True
                        if self.grid[row][column].bomb_count == 0:
                            self.grid[row][column].open_neighbours(self.squares_y, self.squares_x)
                        self.check_victory()
            else:
                self.game_lost = False
                self.reset_game()
        elif button == RIGHT_CLICK and not self.game_won:
            if not self.grid[row][column].is_visible:
                if self.flag_count < self.num_bombs:
                    self.grid[row][column].has_flag = not self.grid[row][column].has_flag
                self.count_flags()



    def draw(self, obj):
        self.width = pygame.display.get_surface().get_width() - 2 * MARGIN
        pygame.draw.rect(screen, GRAY, [MARGIN, 0, self.width, MENU_SIZE])
        self.btn_minus.draw(screen)
        self.btn_plus.draw(screen)
        self.btn_flags.draw(screen)
        self.label_bombs.show(screen, game.num_bombs)
        self.label_flags.show(screen, game.flag_count)
        if obj.game_lost:
            self.label_game_end.show(screen, "Game Over")
        elif obj.game_won:
            self.label_game_end.show(screen, "You Won!")

    class Label:
        def __init__(self, x, y):
            self.x = x
            self.y = y
            self.text = ""

        def show(self, surface, value):
            text = str(value)
            self.text = font.render(text, True, BLACK)
            surface.blit(self.text, (self.x, self.y))


    class Button:
        def __init__(self, x, y, width, height, text, xoff=0, yoff=0):
            self.x = x
            self.y = y
            self.height = height
            self.width = width
            self.background = WHITE
            self.text = text
            self.x_offset = xoff
            self.y_offset = yoff

        def draw(self, surface):
            pygame.draw.ellipse(surface, self.background, [self.x, self.y, self.width, self.height], 0)
            text = font.render(self.text, True, BLACK)
            surface.blit(text, (self.x + self.x_offset, self.y + self.y_offset))

        def click_handle(self):
            pos = pygame.mouse.get_pos()
            if pos[0] > self.x and pos[1] > self.y and pos[0] < (self.x + self.width) and pos[1] < (self.y + self.height):
                return True
            else:
                return False

# Initialize pygame and set screen size and caption
pygame.init()
size = (NSQUARES * (WIDTH + MARGIN) + MARGIN, (NSQUARES * (HEIGHT + MARGIN) + MARGIN) + MENU_SIZE)
screen = pygame.display.set_mode(size, pygame.RESIZABLE)
pygame.display.set_caption("Minesweeper")
# Font to use in the entire game
font = pygame.font.Font('freesansbold.ttf', 24)
# Create instances for Game and Menu
game = Game()
menu = Menu()
clock = pygame.time.Clock()
# Main loop
while True:
    for event in pygame.event.get():
        # Close the game if the user clicks the X
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        # Mouse click events
        elif event.type == pygame.MOUSEBUTTONDOWN:
            # Get mouse position
            position = pygame.mouse.get_pos()
            # Change the screen coordinates to grid coordinates and cap max values
            column = position[0] // (WIDTH + MARGIN)
            row = (position[1] - MENU_SIZE) // (HEIGHT + MARGIN)
            if row >= game.squares_y:
                row = game.squares_y - 1
            if column >= game.squares_x:
                column = game.squares_x - 1
            if row >= 0:
                game.click_handle(row, column, event.button)
            else:
                menu.click_handle(game)
        # Event for screen resize
        elif event.type == pygame.VIDEORESIZE:
            if game.resize:
                game.adjust_grid(event.w, event.h)
                game.reset_game()
            else:
                game.resize = True
    game.draw()
    menu.draw(game)
    clock.tick(60)
    # Update the screen
    pygame.display.flip()


SystemExit: 